# 電影對白爬蟲程式

In [80]:
import urllib.request as ur
from pythonopensubtitles.opensubtitles import OpenSubtitles
import fnmatch
import os
import time
import zipfile
import re
import pickle
from opencc import OpenCC 
#path = '/Users/Wan-Ting/Google Drive/NCTU/NLP/lab2/crawler'
#os.chdir(path)

In [81]:
def t2s(text):
    openCC = OpenCC('tw2sp')  # Traditional Chinese (Taiwan standard) to Simplified Chinese (with phrases)
    to_convert = text
    converted = openCC.convert(to_convert)
    return converted
def s2t(text):
    openCC = OpenCC('s2t')  # convert from Simplified Chinese to Traditional Chinese
    # can also set conversion by calling set_conversion
    # openCC.set_conversion('s2tw')
    to_convert = text
    converted = openCC.convert(to_convert)
    return converted

In [82]:
def download(url,directory_to_extract_to='/Users/Wan-Ting/Downloads/lines'):
    req = ur.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    html = ur.urlopen(req).read()
    #將zip檔另存
    with open("script.zip", 'wb') as f:
        f.write(html)  
        
    with zipfile.ZipFile(open('script.zip', 'rb')) as f:
        FileName = fnmatch.filter(f.namelist(),"*.srt")+fnmatch.filter(f.namelist(),"*.ass")
        if len(FileName)!=0:
            #讀取解壓縮的檔案，並彙整台詞成→new_lines
            for file_name in FileName:
                #資料解壓縮
                f.extract(member=file_name, path=directory_to_extract_to)        
        else:
            print("no .srt script")
    #print(FileName,"downloaded!")
    return FileName

In [83]:
def search(movietitle):
    ost = OpenSubtitles()
    token = ost.login("doctest", 'doctest')
    movie = ost.search_subtitles([{'query' :movietitle, 'sublanguageid' : 'zht'}])
    return movie

In [84]:
def is_time_stamp(l):
    if l[:2].isnumeric() and l[2] == ':':
        return True
    return False


def has_no_text(line):
    #移除分隔符號
    l = line.strip('\n')
    if not len(l):
        return True
    if l.isnumeric():
        return True
    if is_time_stamp(l):
        return True
    if l[0] == '(' and l[-1] == ')':
        return True
    return False

def is_lowercase_letter_or_comma(letter):
    if letter.isalpha() and letter.lower() == letter:
        return True
    if letter == ',':
        return True
    return False

def clean_up(lines):
    new_lines = []
    for line in lines[:]:
        #line = lines[6]
        if has_no_text(line):
            continue

        else:
          #append line
          new_lines.append(line)
    return new_lines

In [85]:
#讀入電影清單(.txt檔)
file = open('/Users/Wan-Ting/Downloads/movie.txt', 'r', encoding = 'utf-8') 
Data = file.read()
Data = Data.splitlines()
file.close()
#將電影名稱全部轉小寫、以''取代'、去除空白以外的符號
#List = [x.lower() for x in Data]
#List = [x.replace("'" ,'') for x in List]
#movietitle_ls = [re.sub(r"[^a-zA-Z0-9]+", ' ', k) for k in List]
#print(List)

In [86]:
movietitle_ls = [line.split('\t') for line in Data[1:]]

In [ ]:
directory_to_extract_to='/Users/Wan-Ting/Downloads/lines'
t = open("0452627_黃婉婷_task1.txt","w")
for mt in movietitle_ls:
    print(movietitle)
    movietitle = mt[0]
    ost = OpenSubtitles()
    token = ost.login("doctest", 'doctest')
    movies = ost.search_subtitles([{'query' :movietitle, 'sublanguageid' : 'zht'}])

    value = 0
    ls = []
    for movie in movies:
        if movie['IDMovieImdb'] == mt[1]:
            if value < int(movie['SubDownloadsCnt']):
                value = int(movie['SubDownloadsCnt'])
                ls.append(movie)
        else:
            pass
            #print('imdbid does not match')
    if len(ls):
        high = ls[-1]
        srt_ls = download(high['ZipDownloadLink'])
        #print('downloaded!')
        #print(srt_ls[0],high['SubEncoding'])
        if high['SubEncoding']=='':
            encoding = 'utf-8'
        else:
            encoding = high['SubEncoding']
        with open(os.path.join(directory_to_extract_to,srt_ls[0]),encoding = encoding, errors="ignore") as f: #
            l = f.readlines()
        num = len(clean_up(l))
        t.write(high['MovieReleaseName']+'\t'+high['SubDownloadLink']+'\t'+str(num)+'\n')
    else:
        print("ls is empty.")

t.close()
print("done!")